In [1]:
!nvidia-smi

Wed May 11 20:13:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q transformers datasets torchinfo rouge_score sacrebleu

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
RANDOM_SEED = 42

# Dataset preparation

We will tokenize the whole data by making use of the `datasets` library, which works seamlessly with the huggingface library.

In [5]:
from datasets import load_dataset, load_from_disk
import os
import sys
import pandas as pd
import csv

DATA_PATH = "/gdrive/MyDrive/final-project/post-refactor/data/"
DATASET_CSV_PATH = os.path.join(DATA_PATH, "balanced_summaries_claims_less_samples.zip")

# set maximum csv size to avoid pandas ram issues
csv.field_size_limit(sys.maxsize)
# load into memory for analysis
df = pd.read_csv(DATASET_CSV_PATH, engine="python").set_index("patentnumber")[["summary", "claim"]]

## Dataset tokenization

Unfortunately, we won't be able to use the whole dataset. That is because BigBird-Pegasus requires a huge amount of memory in order to be finetuned.

To avoid incurring into out-of-memory errors from CUDA we will only use a subset of the available dataset, by taking only those documents whose claim length is shorter than 500 tokens.

In [6]:
summary_len = df.summary.apply(lambda r: len(r.split()))
claim_len = df.claim.apply(lambda r: len(r.split()))

MAX_SUMMARY_LEN = None #@param
MAX_CLAIM_LEN = 500 #@param

subset_clause = summary_len >= 0
if MAX_SUMMARY_LEN is not None:
  subset_clause = subset_clause & (summary_len < MAX_SUMMARY_LEN)
if MAX_CLAIM_LEN is not None:
  subset_clause = subset_clause & (claim_len < MAX_CLAIM_LEN)

subset_df = df[subset_clause]

In [7]:
#@title Tokenize data
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-bigpatent")
TOKENIZED_DATASET_PATH = os.path.join(DATA_PATH, "tokenized_bigbird_dataset_smaller")

# let's check if we can load the dataset from disk first.
# this will save us the burden of loading the tokenizer
# and tokenizing all the data we need
if os.path.exists(TOKENIZED_DATASET_PATH):
  dataset = load_from_disk(TOKENIZED_DATASET_PATH)
  print("Dataset loaded")
else:
  from datasets import Dataset
  dataset = Dataset.from_pandas(subset_df)

  # first let's rename data in the way the model expect
  dataset = dataset.rename_column("summary", "input_ids") \
    .rename_column("claim", "decoder_input_ids")
  # even though we carefully preprocessed data some descriptions are still empty.
  # we will filter them out
  dataset = dataset.filter(lambda r: r["input_ids"] is not None)

  def encoder_tokenize_function(row):
    """
    Tokenize the summary into input_ids and attention_mask
    """
    kwargs = {
        "padding": "max_length",
        "truncation": True,
    }

    if MAX_SUMMARY_LEN is not None:
      kwargs["max_length"] = MAX_SUMMARY_LEN

    return tokenizer(row["input_ids"], **kwargs)

  # tokenize the summaries
  dataset = dataset.map(encoder_tokenize_function, batched=True)

  def decoder_tokenize_function(row):
    """
    Tokenize claim into the expected output from the decoder 
    (decoder_input_ids and decoder_attention_mask)
    """
    kwargs = {
        "padding": "max_length",
        "truncation": True,
    }

    if MAX_CLAIM_LEN is not None:
      kwargs["max_length"] = MAX_CLAIM_LEN

    tokenized = tokenizer(row["decoder_input_ids"], **kwargs)

    return {
        "decoder_input_ids": tokenized["input_ids"],
        "decoder_attention_mask": tokenized["attention_mask"]
    }

  # tokenize the claim
  dataset = dataset.map(decoder_tokenize_function, batched=True)

  def compute_labels(row):
    """
    Compute labels based on decoder_input_ids where padding token is represented as -100
    """
    labels = row["decoder_input_ids"]
    labels = [-100 if t == 0 else t for t in labels]
    return {"labels" : labels}
  
  dataset = dataset.map(compute_labels, batched=True)

  # export the dataset to disk for future loading
  dataset.save_to_disk(TOKENIZED_DATASET_PATH)
  print("Dataset computed and saved")

Dataset loaded


# Dataset splits

We will divide the overall dataset into the usual splits: training and testing respectively $90\%$ and $10\%$ of the overall data.

In [8]:
dataset = dataset.train_test_split(test_size=0.10, seed=RANDOM_SEED)

Loading cached split indices for dataset at /gdrive/MyDrive/final-project/post-refactor/data/tokenized_bigbird_dataset_smaller/cache-22b3f2abe308373b.arrow and /gdrive/MyDrive/final-project/post-refactor/data/tokenized_bigbird_dataset_smaller/cache-2d201d7b99ae91e9.arrow


In [9]:
print(f"Train: {len(dataset['train'])} samples")
print(f"Test: {len(dataset['test'])} samples")

Train: 185 samples
Test: 21 samples


# Neural model

We will use Google's BigbirdPegasus (*BP*) model, by making use of huggingface APIs.

In [10]:
from transformers import BigBirdPegasusForConditionalGeneration
from torchinfo import summary

model = BigBirdPegasusForConditionalGeneration.from_pretrained(
    "google/bigbird-pegasus-large-bigpatent",
    block_size=16,
    num_random_blocks=3,
    attention_type="block_sparse")
model.gradient_checkpointing_enable()
summary(model, dtypes=["torch.IntTensor"])

Layer (type:depth-idx)                                                      Param #
BigBirdPegasusForConditionalGeneration                                      --
├─BigBirdPegasusModel: 1-1                                                  --
│    └─Embedding: 2-1                                                       98,409,472
│    └─BigBirdPegasusEncoder: 2-2                                           --
│    │    └─Embedding: 3-1                                                  (recursive)
│    │    └─BigBirdPegasusLearnedPositionalEmbedding: 3-2                   4,194,304
│    │    └─ModuleList: 3-3                                                 201,474,048
│    │    └─LayerNorm: 3-4                                                  2,048
│    └─BigBirdPegasusDecoder: 2-3                                           --
│    │    └─Embedding: 3-5                                                  (recursive)
│    │    └─BigBirdPegasusLearnedPositionalEmbedding: 3-6                   4,194

In [11]:
from datasets import load_metric
import numpy as np
import nltk

nltk.download('punkt')

ROUGE = load_metric('rouge')
BLEU = load_metric('sacrebleu')

def postprocess_text(preds, labels):
  preds = [pred.strip() for pred in preds]
  labels = [label.strip() for label in labels]

  # rougeLSum expects newline after each sentence
  preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
  labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

  return preds, labels

def compute_metrics(eval_preds):
  preds, labels = eval_preds
  if isinstance(preds, tuple):
      preds = preds[0]

  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  # Replace -100 in the labels to actual padding
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Some simple post-processing
  decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

  rouge_scores = ROUGE.compute(predictions=decoded_preds, 
                               references=[[dl] for dl in decoded_labels])
  rouge_scores = { k: v.mid.fmeasure for k, v in rouge_scores.items() }

  bleu_score = BLEU.compute(predictions=decoded_preds,
                            references=[[dl] for dl in decoded_labels])["score"]
  
  return {"sacrebleu": bleu_score, **rouge_scores}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
FINETUNE_MODEL_PATH = os.path.join(DATA_PATH, "BigBirdModelFineTuneLessSample/")

In [15]:
from transformers import  Seq2SeqTrainingArguments,  Seq2SeqTrainer, EarlyStoppingCallback
from torch.utils.checkpoint import checkpoint 
import gc
gc.collect()

EPOCHS = 3
BATCH_SIZE = 1
ACCUMULATION_STEPS = 2
# perform saving and evaluation every 25% of the dataset
EVAL_LOG_STEPS = int((len(dataset['train']) / (BATCH_SIZE * ACCUMULATION_STEPS)) * 0.25)
EVAL_ACC_STEPS = 1

training_args =  Seq2SeqTrainingArguments(
    output_dir=FINETUNE_MODEL_PATH,
    overwrite_output_dir=True, # used to keep training
    gradient_accumulation_steps=ACCUMULATION_STEPS, # lower memory usage: perform backprop every 2 steps
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    evaluation_strategy="steps",
    eval_steps=EVAL_LOG_STEPS,
    eval_accumulation_steps=EVAL_ACC_STEPS,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_first_step=True,
    logging_strategy="steps",
    logging_steps=EVAL_LOG_STEPS,
    save_strategy="steps",
    save_steps=2 * EVAL_LOG_STEPS,
    save_total_limit=2, # save at most two checkpoints, delete the older ones
    fp16=True, # faster and lighter on memory but possibly less precise on convergence
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="sacrebleu",
    greater_is_better=True,
    gradient_checkpointing=True)

In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model, 
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [19]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BigBirdPegasusForConditionalGeneration.forward` and have been ignored: patentnumber. If patentnumber are not expected by `BigBirdPegasusForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 185
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 276
/usr/local/lib/python3.7/dist-packages/transformers/models/bigbird_pegasus/modeling_bigbird_pegasus.py:805: UserWarning: __floordiv__ is deprecated, and i

Step,Training Loss,Validation Loss,Sacrebleu,Rouge1,Rouge2,Rougel,Rougelsum
23,6.708700,7.045971,25.264330,0.515152,0.417900,0.467806,0.459860
46,6.624800,6.721884,39.302477,0.590481,0.482831,0.523570,0.522859
69,5.570800,6.162240,42.423069,0.592859,0.469750,0.497240,0.497309
92,5.152900,4.984022,41.275319,0.609829,0.488458,0.522106,0.522042
115,4.364200,3.894400,36.816557,0.600010,0.499488,0.531899,0.535502
138,3.677900,2.962279,36.364833,0.577800,0.467990,0.507445,0.506790
161,2.755400,2.153694,39.417138,0.598344,0.470562,0.521861,0.514693
184,2.107500,1.603390,39.976869,0.627739,0.510491,0.552158,0.554787
207,1.769000,1.263733,39.172256,0.625257,0.525342,0.569315,0.565941
230,1.471700,1.000870,38.379916,0.611600,0.497168,0.542629,0.544545


The following columns in the evaluation set  don't have a corresponding argument in `BigBirdPegasusForConditionalGeneration.forward` and have been ignored: patentnumber. If patentnumber are not expected by `BigBirdPegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 21
  Batch size = 1


The following columns in the evaluation set  don't have a corresponding argument in `BigBirdPegasusForConditionalGeneration.forward` and have been ignored: patentnumber. If patentnumber are not expected by `BigBirdPegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 21
  Batch size = 1
Saving model checkpoint to /gdrive/MyDrive/final-project/post-refactor/data/BigBirdModelFineTuneLessSample/checkpoint-46
Configuration saved in /gdrive/MyDrive/final-project/post-refactor/data/BigBirdModelFineTuneLessSample/checkpoint-46/config.json
Model weights saved in /gdrive/MyDrive/final-project/post-refactor/data/BigBirdModelFineTuneLessSample/checkpoint-46/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/bigbird_pegasus/modeling_bigbird_pegasus.py:805: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trun

ValueError: ignored